In [ ]:
import pandas as pd
import pickle
import networkx as nx
import plotly.graph_objects as go
import matplotlib.pyplot as plt
import os.path

In [ ]:
cache_file = r'd:\users\colomikunoj\Desktop\Github 3GPP Meeting Helper\3gpp-meeting-tools\3GPP Meeting Email Statistics\TDocsByAgenda_TSGS2_141e_Electronic_24200b6d2486c6799f68217fa19aa136.pickle'

In [ ]:
with open(cache_file, 'rb') as f:
    # Unpickle the 'data' dictionary using the highest protocol available.
    cache               = pickle.load(f)
    tdocs               = cache['tdocs']
    contributor_columns = cache['contributor_columns']
    others_cosigners    = cache['others_cosigners']

In [ ]:
contributors = [e.replace('Contributed by ','') for e in contributor_columns]
contributors.append('Others')
pd.options.display.max_columns = None
pd.options.display.max_rows    = 60
pd.options.display.min_rows    = 15
display(tdocs)

In [ ]:
G = nx.Graph()
contributors_in_meeting = []
nodes = {}
edge_sizes = {}
for idx,contributor in enumerate(contributors):
    # Each contributor is a node
    contributor_size = tdocs[(tdocs['Contributed by {0}'.format(contributor)]==True)].count()['AI']
    if contributor_size > 0:
        contributors_in_meeting.append(contributor)
        nodes[contributor] = contributor_size
        G.add_node(contributor, size=contributor_size)

total_contributors = len(contributors_in_meeting)
for idx,contributor_1 in enumerate(contributors_in_meeting):
    for contributor_2 in contributors_in_meeting[idx+1:total_contributors]:
        tdoc_count = tdocs[(tdocs['Contributed by {0}'.format(contributor_1)]==True) & (tdocs['Contributed by {0}'.format(contributor_2)]==True)].count()['AI']
        if tdoc_count > 0:
            edge_sizes['{0}-{1}'.format(contributor_1,contributor_2)] = tdoc_count
            G.add_edge(contributor_1, contributor_2, weight=tdoc_count)
            # print('{0}, {1}: {2}'.format(contributor, contributor_2, tdoc_count))

In [ ]:
[cache_folder, picke_file] = os.path.split(cache_file)
graphml_path = os.path.join(cache_folder, '{0}.graphml'.format(picke_file.split('.')[0]))
nx.write_graphml(G, graphml_path)
display(graphml_path)

In [ ]:
edge_sizes

In [ ]:
display(tdocs.Result.unique())
display(tdocs.Type.unique())

In [ ]:
# We will show for the following document types:
#  - P-CR
#  - CR
#  - LS In
#  - LS OUT
# For each of the categories, a document can end in either Merged/Revised, Withdrawn, Unhandled, Approved/Agreed
# Left we have initial documents, middle are merged/revisions (some may be repeated, just a sum of what was revised/merged from the left part), left final documents